# Train a BERT classifier

**TO DO:** To use it: https://github.com/charles9n/bert-sklearn

In [ ]:
import pickle
from bert_sklearn import BertClassifier
from bert_sklearn import load_model
import pandas as pd
import pathlib
from tools import processing

import spacy

nlp = spacy.load("en_core_web_sm")

model = BertClassifier()    

#### Load data

In [ ]:
corpus = "stories/"
dataset_df = pd.read_pickle("../data/" + corpus + "train.pkl")

In [ ]:
dataset_df.columns

#### Train BERT classifiers

In [ ]:
dFolders = {"targetExpression": "../models/classifiers/" + corpus + "targetExpression/",
            "context3wmasked": "../models/classifiers/" + corpus + "context3wmasked/",
            "context3w": "../models/classifiers/" + corpus + "context3w/"}

for col in dFolders:
    
    if not pathlib.Path(dFolders[col] + "/bert.bin").exists():
        pathlib.Path(dFolders[col]).mkdir(parents=True, exist_ok=True)

        X = dataset_df[col].tolist()
        y = dataset_df["animated"].tolist()

        print (len(X), len(y))

        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        # finetune model
        model.fit(X_train, y_train)

        # make predictions
        y_pred = model.predict(X_test)

        from sklearn.metrics import precision_recall_fscore_support

        precision_recall_fscore_support(y_pred, y_test, average='macro')

        # save model to disk
        savefile= dFolders[col] + "/bert.bin"
        model.save(savefile)
        print("DONE")
        print()